In [35]:
import numpy as np
import math

## 1.1 Ejercicios Básicos

### 1 Procesamiento de visibilidades

In [57]:
# 1.1 Ejercicios Básicos
## 1 Procesamiento de visibilidades
def visilities(n=1_000_000, dtype=np.int32):
    amplitud = np.random.uniform(0, 1, n).astype(dtype)
    fase = np.random.uniform(0, 2 * np.pi, n).astype(dtype)
    visibilidades = amplitud * np.exp(1j * fase)
    return visibilidades


def prom_power(v):
    prom = []
    for value in v:
        real = value.real
        imag = value.imag
        p = math.sqrt(pow(real,2) + pow(imag,2))
        prom.append(p)

    sum = 0 
    for n in prom:
        sum += n
    prom = sum / len(v)
    return prom

def prom_power_vectorized(v):
    return np.mean(np.abs(v))

In [55]:
%%time
v = visilities()

CPU times: total: 31.2 ms
Wall time: 35.7 ms


In [46]:
%%time
prom = prom_power(v)

CPU times: total: 281 ms
Wall time: 285 ms


In [47]:
%%time
prom_np = prom_power_vectorized(v)

CPU times: total: 0 ns
Wall time: 2.67 ms


### 2 Optimizacion de memoria en Datos astronómicos

In [65]:
## 2 Optimizacion de memoria en Datos astronómicos
v_int32 = visilities(1000)
v_int64 = visilities(1000, dtype=np.int64)
v_float32 = visilities(1000, dtype=np.float32)
v_float64 = visilities(1000,dtype=np.float64)

print(f"Memoria I32: {v_int32.nbytes / 1e6} MB")
print(f"Memoria I64: {v_int64.nbytes / 1e6} MB")
print(f"Memoria F32: {v_float32.nbytes / 1e6} MB")
print(f"Memoria F64: {v_float64.nbytes / 1e6} MB")

Memoria I32: 0.016 MB
Memoria I64: 0.016 MB
Memoria F32: 0.008 MB
Memoria F64: 0.016 MB


In [66]:
ganancia = 2